In [49]:
import pandas as pd
import numpy as np

In [50]:
df=pd.read_csv('covid_toy.csv')

In [51]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [52]:
df['gender'].value_counts(),df['city'].value_counts(),df['cough'].value_counts()

(gender
 Female    59
 Male      41
 Name: count, dtype: int64,
 city
 Kolkata      32
 Bangalore    30
 Delhi        22
 Mumbai       16
 Name: count, dtype: int64,
 cough
 Mild      62
 Strong    38
 Name: count, dtype: int64)

In [53]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [54]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2,random_state=0)

x_train.shape

(80, 5)

# when you dont know about the transformer you have to implement all the encoders acc to their catgories differently then combine them together

In [55]:
from sklearn.impute import SimpleImputer           # fever column vich missing values hegiya si jo eh mean nal fill karega 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [56]:
# simple imputer to fever 
si=SimpleImputer()
#train te kita
x_train_fever=si.fit_transform(x_train[['fever']])   # double bracket return the dataframe and single series

#also test the data
x_test_fever=si.transform(x_test[['fever']])

x_train_fever.shape


(80, 1)

In [57]:
#ordinal encoder to cough

oe=OrdinalEncoder(categories=[['Mild','Strong']])
# train te kita
x_train_cough=oe.fit_transform(x_train[['cough']])
# test te kita
x_test_cough=oe.transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [58]:
# one hot encoder to gender,city

ohe=OneHotEncoder(drop='first',sparse_output=False)
# train te kita
x_train_gender_city=ohe.fit_transform(x_train[['gender','city']])

# test te kita
x_test_gender_city=ohe.transform(x_test[['gender','city']])

x_train_gender_city.shape

(80, 4)

In [59]:
# extracting the age column last feature left 
x_train_age=x_train.drop(columns=['gender','fever','cough','city']).values

x_test_age=x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [60]:
x_train_transformed=np.concatenate((x_train_gender_city,x_train_fever,x_train_cough,x_train_age),axis=1)

x_test_transformed=np.concatenate((x_test_gender_city,x_test_fever,x_test_cough,x_test_age),axis=1)

x_train_transformed.shape

(80, 7)

# if you know transformer

In [61]:
from sklearn.compose import ColumnTransformer

# transformer=ColumnTransformer(transformers=[] , remainder='')     blueprint

transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]) , ['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
] , remainder='passthrough')  # passthrough means trannsformers ch jeda column nhi pass hoya ohnu oh undekha kardega tusi drop vi use kar sakde si 


In [62]:
transformer.fit_transform(x_train)

array([[ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  22.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  56.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  31.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  75.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  72.        ],
       [ 99.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  66.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  14.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  10.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          1.    

In [63]:
transformer.fit_transform(x_train).shape

(80, 7)

In [64]:
transformer.transform(x_test).shape

(20, 7)